# Feed Forward Neural Network MNIST

### Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Hyperparameters

In [2]:
input_size = 784        # Number of input neurons (image pixels) 28 x 28
hidden_size = 400       # Number of hidden neurons
out_size = 10           # Number of classes (0-9) 
epochs = 10             # How many times we pass our entire dataset into our network 
batch_size = 100        # Input size of the data during one iteration 
learning_rate = 0.001   # How fast we are learning

### Dataset

In [3]:
train_dataset = datasets.MNIST(
    root='./data', 
    train=True, 
    transform=transforms.ToTensor(), 
    download=True
)

test_dataset = datasets.MNIST(
    root='./data',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



### Data Loader

In [4]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

### Model

In [5]:
class FeedForwardMNIST(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # hidden layer 1
        self.fc2 = nn.Linear(hidden_size, hidden_size) # hidden layer 2
        self.fc3 = nn.Linear(hidden_size, out_size) # output layer
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

### Model Object

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FeedForwardMNIST(input_size, hidden_size, out_size).to(device)
criteria = nn.CrossEntropyLoss() # Softmax is internally computed so we don't need to add it
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training

In [7]:
for epoch in range(epochs):
    correct = 0
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        # Flatten the image and move to GPU if available
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        # Get predictions from the maximum value
        # It returns the maximum value and the index of the maximum value. We only need the index.
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        # Compute loss
        loss = criteria(outputs, labels)
        running_loss += loss.item()
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {correct/len(train_dataset):.4f}')

Epoch [1/10], Loss: 0.2629, Accuracy: 0.9232
Epoch [2/10], Loss: 0.0959, Accuracy: 0.9707
Epoch [3/10], Loss: 0.0613, Accuracy: 0.9809
Epoch [4/10], Loss: 0.0437, Accuracy: 0.9863
Epoch [5/10], Loss: 0.0341, Accuracy: 0.9886
Epoch [6/10], Loss: 0.0246, Accuracy: 0.9918
Epoch [7/10], Loss: 0.0222, Accuracy: 0.9926
Epoch [8/10], Loss: 0.0193, Accuracy: 0.9933
Epoch [9/10], Loss: 0.0160, Accuracy: 0.9947
Epoch [10/10], Loss: 0.0143, Accuracy: 0.9954


### Testing

In [8]:
with torch.no_grad():
    correct = 0
    for images, labels in test_loader:
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {correct/len(test_dataset):.4f}')

Accuracy of the network on the 10000 test images: 0.9811
